<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [14]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [15]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Yvonne", "transactions": [{"transaction-id": 881, "amount": 2839}, {"transaction-id": 1881, "amount": 3021}, {"transaction-id": 6299, "amount": 2887}, {"transaction-id": 6556, "amount": 2674}, {"transaction-id": 7151, "amount": 2993}, {"transaction-id": 8203, "amount": 2765}, {"transaction-id": 11161, "amount": 2826}, {"transaction-id": 15544, "amount": 2756}, {"transaction-id": 17482, "amount": 2912}, {"transaction-id": 20212, "amount": 2780}, {"transaction-id": 20588, "amount": 2815}, {"transaction-id": 21835, "amount": 2737}, {"transaction-id": 28893, "amount": 2937}, {"transaction-id": 40956, "amount": 2721}, {"transaction-id": 41099, "amount": 2968}, {"transaction-id": 46778, "amount": 2845}, {"transaction-id": 51725, "amount": 2973}, {"transaction-id": 55878, "amount": 3065}, {"transaction-id": 67514, "amount": 2812}, {"transaction-id": 74122, "amount": 2704}, {"transaction-id": 74532, "amount": 2921}, {"transaction-id": 75504, "amount": 2581}, {"transaction-

In [17]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 storage_options={'anon': True})
b.take(2)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',
 '2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896484375,40.750110626220703,1,N,-73.974784851074219,40.750617980957031,1,12,1,0.5,3.25,0,0.3,17.05\n')

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [18]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [19]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [20]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Yvonne", "transactions": [{"transaction-id": 881, "amount": 2839}, {"transaction-id": 1881, "amount": 3021}, {"transaction-id": 6299, "amount": 2887}, {"transaction-id": 6556, "amount": 2674}, {"transaction-id": 7151, "amount": 2993}, {"transaction-id": 8203, "amount": 2765}, {"transaction-id": 11161, "amount": 2826}, {"transaction-id": 15544, "amount": 2756}, {"transaction-id": 17482, "amount": 2912}, {"transaction-id": 20212, "amount": 2780}, {"transaction-id": 20588, "amount": 2815}, {"transaction-id": 21835, "amount": 2737}, {"transaction-id": 28893, "amount": 2937}, {"transaction-id": 40956, "amount": 2721}, {"transaction-id": 41099, "amount": 2968}, {"transaction-id": 46778, "amount": 2845}, {"transaction-id": 51725, "amount": 2973}, {"transaction-id": 55878, "amount": 3065}, {"transaction-id": 67514, "amount": 2812}, {"transaction-id": 74122, "amount": 2704}, {"transaction-id": 74532, "amount": 2921}, {"transaction-id": 75504, "amount": 2581}, {"transaction-

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [21]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Yvonne',
  'transactions': [{'transaction-id': 881, 'amount': 2839},
   {'transaction-id': 1881, 'amount': 3021},
   {'transaction-id': 6299, 'amount': 2887},
   {'transaction-id': 6556, 'amount': 2674},
   {'transaction-id': 7151, 'amount': 2993},
   {'transaction-id': 8203, 'amount': 2765},
   {'transaction-id': 11161, 'amount': 2826},
   {'transaction-id': 15544, 'amount': 2756},
   {'transaction-id': 17482, 'amount': 2912},
   {'transaction-id': 20212, 'amount': 2780},
   {'transaction-id': 20588, 'amount': 2815},
   {'transaction-id': 21835, 'amount': 2737},
   {'transaction-id': 28893, 'amount': 2937},
   {'transaction-id': 40956, 'amount': 2721},
   {'transaction-id': 41099, 'amount': 2968},
   {'transaction-id': 46778, 'amount': 2845},
   {'transaction-id': 51725, 'amount': 2973},
   {'transaction-id': 55878, 'amount': 3065},
   {'transaction-id': 67514, 'amount': 2812},
   {'transaction-id': 74122, 'amount': 2704},
   {'transaction-id': 74532, 'amount': 2

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [22]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 2,
  'name': 'Alice',
  'transactions': [{'transaction-id': 178, 'amount': 973},
   {'transaction-id': 302, 'amount': 766},
   {'transaction-id': 445, 'amount': 899},
   {'transaction-id': 1132, 'amount': 760},
   {'transaction-id': 1476, 'amount': 894},
   {'transaction-id': 1827, 'amount': 955},
   {'transaction-id': 2041, 'amount': 746},
   {'transaction-id': 3598, 'amount': 787},
   {'transaction-id': 4176, 'amount': 902},
   {'transaction-id': 5205, 'amount': 781},
   {'transaction-id': 5311, 'amount': 849},
   {'transaction-id': 5631, 'amount': 878},
   {'transaction-id': 5716, 'amount': 900},
   {'transaction-id': 5931, 'amount': 971},
   {'transaction-id': 6028, 'amount': 795},
   {'transaction-id': 6113, 'amount': 707},
   {'transaction-id': 6643, 'amount': 777},
   {'transaction-id': 7237, 'amount': 802},
   {'transaction-id': 7458, 'amount': 913},
   {'transaction-id': 7912, 'amount': 947},
   {'transaction-id': 8083, 'amount': 700},
   {'transaction-id': 8264, 'amou

In [23]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 341},
 {'name': 'Alice', 'count': 6},
 {'name': 'Alice', 'count': 433},
 {'name': 'Alice', 'count': 98},
 {'name': 'Alice', 'count': 14})

In [36]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(341, 6, 433, 98, 14)

In [48]:
%%time
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

CPU times: user 344 ms, sys: 29.2 ms, total: 374 ms
Wall time: 2.34 s


188.49466666666666

In [49]:
%%time

sanity_check = (
    js.filter(lambda r: r["name"] == "Alice")
    .map(count_transactions)
    .pluck("count")
    .compute()
)

sum(sanity_check) / len(sanity_check)

CPU times: user 290 ms, sys: 28.2 ms, total: 318 ms
Wall time: 2.13 s


188.49466666666666

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [50]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 178, 'amount': 973},
  {'transaction-id': 302, 'amount': 766},
  {'transaction-id': 445, 'amount': 899},
  {'transaction-id': 1132, 'amount': 760},
  {'transaction-id': 1476, 'amount': 894},
  {'transaction-id': 1827, 'amount': 955},
  {'transaction-id': 2041, 'amount': 746},
  {'transaction-id': 3598, 'amount': 787},
  {'transaction-id': 4176, 'amount': 902},
  {'transaction-id': 5205, 'amount': 781},
  {'transaction-id': 5311, 'amount': 849},
  {'transaction-id': 5631, 'amount': 878},
  {'transaction-id': 5716, 'amount': 900},
  {'transaction-id': 5931, 'amount': 971},
  {'transaction-id': 6028, 'amount': 795},
  {'transaction-id': 6113, 'amount': 707},
  {'transaction-id': 6643, 'amount': 777},
  {'transaction-id': 7237, 'amount': 802},
  {'transaction-id': 7458, 'amount': 913},
  {'transaction-id': 7912, 'amount': 947},
  {'transaction-id': 8083, 'amount': 700},
  {'transaction-id': 8264, 'amount': 561},
  {'transaction-id': 8861, 'amount': 1001},
  {'transacti

In [51]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 178, 'amount': 973},
 {'transaction-id': 302, 'amount': 766},
 {'transaction-id': 445, 'amount': 899})

In [52]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(973, 766, 899)

In [53]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

639.1927764534452

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [54]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [57]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [58]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [63]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [64]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 180), ('Alice', 180), ('Alice', 195), ('Alice', 195), ('Bob', 241), ('Bob', 241), ('Bob', 260), ('Bob', 260), ('Charlie', 252), ('Charlie', 252), ('Charlie', 271), ('Charlie', 273), ('Dan', 192), ('Dan', 192), ('Dan', 208), ('Dan', 208), ('Edith', 240), ('Edith', 260), ('Edith', 500), ('Frank', 323), ('Frank', 324), ('Frank', 351), ('Frank', 351), ('George', 264), ('George', 264), ('George', 286), ('George', 286), ('Hannah', 180), ('Hannah', 180), ('Hannah', 193), ('Hannah', 195), ('Ingrid', 156), ('Ingrid', 156), ('Ingrid', 169), ('Ingrid', 169), ('Jerry', 300), ('Jerry', 300), ('Jerry', 325), ('Jerry', 325), ('Kevin', 144), ('Kevin', 144), ('Kevin', 156), ('Kevin', 156), ('Laura', 216), ('Laura', 216), ('Laura', 234), ('Laura', 234), ('Michael', 311), ('Michael', 312), ('Michael', 335), ('Michael', 337), ('Norbert', 299), ('Norbert', 299), ('Norbert', 321), ('Norbert', 322), ('Oliver', 364), ('Oliver', 364), ('Oliver', 672), ('Patricia', 216), ('Patricia', 216), ('Patricia

In [78]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 750), ('Bob', 1002), ('Charlie', 1048), ('Dan', 800), ('Edith', 1000), ('Frank', 1349), ('George', 1100), ('Hannah', 748), ('Ingrid', 650), ('Jerry', 1250), ('Kevin', 600), ('Laura', 900), ('Michael', 1295), ('Norbert', 1241), ('Oliver', 1400), ('Patricia', 900), ('Quinn', 1000), ('Ray', 547), ('Sarah', 1150), ('Tim', 897), ('Ursula', 1099), ('Victor', 889), ('Wendy', 450), ('Xavier', 1300), ('Yvonne', 850), ('Zelda', 650)]
CPU times: user 239 ms, sys: 20.3 ms, total: 259 ms
Wall time: 1.74 s


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [95]:
%%time
# Your code here...

# val is like: {
#     'name': 'Alice',
#     'transactions': [
#         {'amount': 1, 'id': 123},
#         {'amount': 2, 'id': 456}
#     ]
# }

def binop(tot, val):
    """
    'tot' will first be the value we set in 'initial'
    'val' will be 1 record from 'js' (picture a list of dictionaries)
    
    Doing something like this triggers an error:
        "daemonic processes are not allowed to have children"
        return tot + db.from_sequence(val["transactions"]).pluck("amount").sum().compute()
    """
    amt = 0
    for t in val["transactions"]:
        amt += t["amount"]
    return tot + amt

(
    js.foldby("name", binop=binop, initial=0, combine=add, combine_initial=0)
    .compute()
)

CPU times: user 140 ms, sys: 45.9 ms, total: 186 ms
Wall time: 1.53 s


[('Yvonne', 170912663),
 ('Oliver', 94042669),
 ('Alice', 90363322),
 ('Xavier', 110206828),
 ('Michael', 166644411),
 ('Victor', 90866187),
 ('Frank', 316767258),
 ('Zelda', 96944086),
 ('Bob', 79818848),
 ('Hannah', 19866823),
 ('Dan', 150676096),
 ('Ingrid', 185979991),
 ('Norbert', 223018492),
 ('Wendy', 148528992),
 ('Patricia', 231115620),
 ('Tim', 402421148),
 ('Laura', 351349733),
 ('Edith', 53600178),
 ('George', 164667722),
 ('Quinn', 46883113),
 ('Jerry', 74525097),
 ('Charlie', 130762374),
 ('Sarah', 231508840),
 ('Ursula', 330555265),
 ('Kevin', 21066055),
 ('Ray', 68228392)]

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [79]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Yvonne,"[{'transaction-id': 881, 'amount': 2839}, {'tr..."
1,1,Oliver,"[{'transaction-id': 2191, 'amount': 1129}, {'t..."
2,2,Alice,"[{'transaction-id': 178, 'amount': 973}, {'tra..."
3,3,Xavier,"[{'transaction-id': 256, 'amount': -480}, {'tr..."
4,4,Michael,"[{'transaction-id': 1996, 'amount': 1435}, {'t..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [82]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 276 ms, sys: 27.7 ms, total: 304 ms
Wall time: 2.05 s


name
Alice       750
Bob        1002
Charlie    1048
Dan         800
Edith      1000
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [83]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Yvonne', 'amount': 2839, 'transaction-id': 881},
 {'id': 0, 'name': 'Yvonne', 'amount': 3021, 'transaction-id': 1881},
 {'id': 0, 'name': 'Yvonne', 'amount': 2887, 'transaction-id': 6299})

In [84]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Yvonne,2839,881
1,0,Yvonne,3021,1881
2,0,Yvonne,2887,6299
3,0,Yvonne,2674,6556
4,0,Yvonne,2993,7151


In [85]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 576 ms, sys: 42.8 ms, total: 619 ms
Wall time: 4.23 s


name
Alice       141371
Bob         198240
Charlie     297757
Dan         246073
Edith       130608
Frank       303300
George      218591
Hannah      132115
Ingrid      136305
Jerry       178332
Kevin        63630
Laura       240743
Michael     305758
Norbert     277970
Oliver      192942
Patricia    201550
Quinn       156620
Ray         167207
Sarah       205829
Tim         237060
Ursula      209321
Victor      208459
Wendy        81949
Xavier      182976
Yvonne      148462
Zelda       136832
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [86]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
